In [ ]:
from google.colab import drive

In [ ]:
import pandas as pd

In [ ]:
# 데이터 파일 path 선언
colab_path = "gdrive/My Drive/AI Security Project/"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pandas.plotting import scatter_matrix

# Modeling

In [ ]:
train_6d_pca = pd.read_csv(colab_path +'train_6d_pca.csv')
test_6d_pca = pd.read_csv(colab_path + 'test_6d_pca.csv')

train_y =  pd.read_csv(colab_path +'train_y.csv')
test_y =  pd.read_csv(colab_path +'test_y.csv')

# PCA 진행

In [ ]:
# 정답 추출과 숫자형 데이터만 따로 남기고 PCA 진행
train_y = strat_train_set['type']
test_y = strat_test_set['type']
strat_train_set = strat_train_set.drop('type', axis=1)
strat_test_set = strat_test_set.drop('type', axis=1)

In [ ]:
# 앙상블을 위한 일반 PCA 진행
from sklearn.decomposition import PCA
pca = PCA(6)

# pca 진행
pca.fit(data_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)

# PCA 결과 추출된 데이터
data_prepared = pca.fit_transform(data_scaled)
test_prepared = pca.transform(test_scaled)

In [ ]:
# 몇 차원 정도 나왔는지 확인
data_prepared.shape

(690444, 6)

In [ ]:
# # 앙상블 점진 PCA 코드는 아래를 실행하자.
# ## 점진 PCA란, 전체 데이터를 한꺼번에 고려해서 차원 축소를 하는 것이 아니라, 몇 가지 샘플을 바탕으로 차원을 축소하는 batch 방식이다.
# ## 점진 PCA는 훈련 셋의 볼륨이 클 때 유용하게 사용할 수 있다.
# from sklearn.decomposition import IncrementalPCA

# n_batches = 100
# inc_pca = IncrementalPCA(6) # 설명된 분산 0.95에 맞는 6으로 설정
# for X_batch in np.array_split(strat_train_set, n_batches):
#   inc_pca.partial_fit(X_batch)

# data_prepared = inc_pca.transform(strat_train_set)
# test_prepared = inc_pca.transform(strat_test_set)

# Modeling

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
# RandomForestClassifier 학습
from sklearn.ensemble import RandomForestClassifier

best_rfc = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42,
                                  max_depth = 65
                                  )
best_rfc.fit(data_prepared, train_y)

RandomForestClassifier(max_depth=65, n_jobs=-1, oob_score=True, random_state=42)

In [ ]:
# F1 Score 검사, 정밀도 및 재현도 검사
from sklearn.metrics import f1_score, precision_score, recall_score

print(best_rfc.score(data_prepared, train_y))
print(best_rfc.score(test_prepared, test_y))
print(f1_score(test_y, best_rfc.predict(test_prepared), average='weighted'))
print(precision_score(test_y, best_rfc.predict(test_prepared), average='weighted'))
print(recall_score(test_y, best_rfc.predict(test_prepared), average='weighted'))

0.9999985516566151
0.9982967580469492
0.9982951650251128
0.9982954824374075
0.9982967580469492


In [ ]:
# RandomForest Classifier 하이퍼 파라미터 탐색
## 해당 하이퍼 파라미터 탐색은 Incremental PCA로 차원 축소된 데이터를 학습한 RandomForestClassifier를 대상으로 함.
from sklearn.model_selection import GridSearchCV
params = {
    'max_depth':[35, 45, 55, 65, 75],        
          }
gs = GridSearchCV(RandomForestClassifier(random_state=42), params, n_jobs=-1)
gs.fit(data_prepared, train_y)
gs.best_params_

KeyboardInterrupt: ignored

In [ ]:
# DecisionTree 학습
from sklearn.tree import DecisionTreeClassifier
gbf = DecisionTreeClassifier()
gbf.fit(data_prepared, train_y)
train_score = gbf.score(data_prepared, train_y)
test_score = gbf.score(test_prepared, test_y)

In [ ]:
# DecisionTree 성능 평가
f1_scores = f1_score(test_y, gbf.predict(test_prepared), average='weighted')
precision_score = precision_score(test_y, gbf.predict(test_prepared), average='weighted')
recall_score = recall_score(test_y, gbf.predict(test_prepared), average='weighted')

print("Train Score : ", train_score)
print("Test Score:", test_score)
print("F1 score : ", f1_scores)
print("precision_score : ", precision_score)
print("recall_score : ", recall_score)